## 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드


### 1단계: 문서 분할 설정 

In [18]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_path = "C:\mylangchain\langchain_basic\data\콘텐츠분쟁해결_사례.pdf"

loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

split_docs = text_splitter.split_documents(documents)

print(f"분할된 청크 수: {len(split_docs)}")
print(split_docs[0].page_content[:500])


<>:4: SyntaxWarning: invalid escape sequence '\m'
<>:4: SyntaxWarning: invalid escape sequence '\m'
C:\Users\user\AppData\Local\Temp\ipykernel_2012\691421458.py:4: SyntaxWarning: invalid escape sequence '\m'
  pdf_path = "C:\mylangchain\langchain_basic\data\콘텐츠분쟁해결_사례.pdf"


분할된 청크 수: 104
콘텐츠분쟁조정 법리 연구 2부
- 타 분쟁조정사례 조사 -


### 2단계: 임베딩 모델 설정 

In [19]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
    dimensions=1536
)

vectorstore = FAISS.from_documents(split_docs, embedding)

vectorstore.save_local("rag_index/contents_law")


In [20]:
vectorstore = FAISS.load_local(
    "rag_index/contents_law",
    embeddings=embedding,
    allow_dangerous_deserialization=True  # ✔ 반드시 추가!
)


### 3단계: 검색기 설정 

In [21]:
retriever = vectorstore.as_retriever(
    search_type="similarity",     # 또는 "mmr" (다양성 고려)
    search_kwargs={"k": 5}        # 관련 문서 5개 반환
)


In [22]:
question = "온라인 강의를 중도 해지하려는데 위약금이 과도합니다. 어떻게 해결할 수 있나요?"

docs = retriever.get_relevant_documents(question)

for i, doc in enumerate(docs):
    print(f"\n--- 관련 문서 {i+1} ---")
    print(doc.page_content[:500])  # 각 문서 일부 출력



--- 관련 문서 1 ---
2007_ 온라인통신교육서비스 대금 환급 요구2················································· 5 5
2007_ 인터넷교육서비스 중도해지 요구···························································· 5 7
2008_ 온라인 통신교육서비스 이용료 환급 요구·············································· 5 9
2008_ 온라인통신교육서비스 이용료 환급 요구2·············································· 6 1
2009_ 인터넷 학습사이트 가입 해지에 따른 대금 환급 요구··························· 6 3
2010_ 화상불량으로 중도 해지한 인터넷강의 환급 요구·································· 6 5
2010_ 인터넷 화상 강의 수강계약 해지에 따른 환급 요구········

--- 관련 문서 2 ---
콘텐츠분쟁조정 법리 연구 2부 - 타 분쟁조정사례 조사 -
88■ 2-다 전자거래분쟁조정위원회 조정결정례
2008_온라인 동영상 강의 계약 해지 청구
【사건개요 】
신청인은 2008. 4. 2. 피신청인의 사이트에서 강의기간 120일 일시정지 30일을 계약내용으로 하는 
온라인 강좌를 2008. 4. 12일부터 시작하기로 458,500 원에 계약하였다 . 그러나 신청인 개인의 사정
으로 강의 시작일을 연기하였다 . 5. 12.이후로도 강의를 듣지 못한 신청인은 2008. 6. 23. 피신청인 
사이트 Q&A게시판을 통해 계약 해지 의사를 표명하였으나 피신청인은 웹사이트 수강료 환불기준은 
강의를 수강한 지 일주일 이내에 신청한 강의의 수강률이 1/3미만일 경우에 한다고 규정하는 것을 
근거로 하여 환불기간이 지났음을 이유로 환불이 불가능함을 주장하면서 분쟁이 발생하였다 .
【쟁점사항 】
 · 수강하지 않은 온라인 강좌에 

### 4단계: LLM 설정 

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # OpenAI 쓸 경우 생략
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.2,
    max_tokens=2000
)


### 5단계: 법률 자문 프롬프트 작성

In [24]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 자문:
"""
)


### 6단계: QA 체인 생성

In [25]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)


### 7단계: 테스트 질문 작성 

In [26]:
test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]


### 8단계: 분쟁 유형 분류 함수 ( 선택 )

In [27]:
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"
